In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import col, sum, count, count_distinct
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second, datediff



spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/13 06:00:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/13 06:00:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
test = spark.read.table("warehouse_dev.silver.orders.diff_delivery_date")
test.show()

+--------------------+--------------------+------------------+-------+
|            order_id|         customer_id|diff_delivery_date|is_late|
+--------------------+--------------------+------------------+-------+
|acce194856392f074...|7e20bf5ca92da6820...|                32|  false|
|1d067305b599c1e0d...|0489975a325480c9e...|                 0|  false|
|6f841dde94727854e...|a9c9532060c9d245f...|                 9|  false|
|ca290a06ee0945b95...|acd575d7382968889...|                15|  false|
|3735720b8c1cd4212...|94af59d9cac1ae197...|                30|  false|
|8b346abc34a6e64bc...|dd1506d329d9b0135...|                14|  false|
|a4d7c8bca45b56444...|fcdb673d0f8d2b84d...|                14|  false|
|b64b1539563ff5159...|a22203b8cd7210764...|                28|  false|
|e05ad3bb40dd7a1f0...|348dab3f7710f2be3...|                16|  false|
|0c00bf3feaaced2a1...|a4f809d1f8681ff7b...|                14|  false|
|69b4682d3ab5ef1f1...|765b56e9a7716dfe3...|                 4|  false|
|f222c

In [3]:
orders = spark.read.table("dev.silver.olist_orders_dataset")
orders.show(truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |2018-06-04 00:00:13     |2018-06-05 00:35:10|2018-06-05 13:24:00         |2018-06-16 15:20:55          |2018-07-18 00:00:00          |
|1d067305b599c1e0dceb3864056ea527|0489975a325480c9e385e9f135bb13c3|delivered   |2018-02-14 13:05:17     |2018-02-14 13:15:38|2018-02-20 20:1

In [ ]:
diff_delievery = orders.select(
    'order_id', 'customer_id',
    datediff(orders['order_estimated_delivery_date'], orders['order_delivered_customer_date']).alias("diff_delievery")
    )
diff_delievery.show()

+--------------------+--------------------+--------------+
|            order_id|         customer_id|diff_delievery|
+--------------------+--------------------+--------------+
|acce194856392f074...|7e20bf5ca92da6820...|            32|
|1d067305b599c1e0d...|0489975a325480c9e...|             0|
|6f841dde94727854e...|a9c9532060c9d245f...|             9|
|ca290a06ee0945b95...|acd575d7382968889...|            15|
|3735720b8c1cd4212...|94af59d9cac1ae197...|            30|
|8b346abc34a6e64bc...|dd1506d329d9b0135...|            14|
|a4d7c8bca45b56444...|fcdb673d0f8d2b84d...|            14|
|b64b1539563ff5159...|a22203b8cd7210764...|            28|
|e05ad3bb40dd7a1f0...|348dab3f7710f2be3...|            16|
|0c00bf3feaaced2a1...|a4f809d1f8681ff7b...|            14|
|69b4682d3ab5ef1f1...|765b56e9a7716dfe3...|             4|
|f222c56f035b47dfa...|b74ca180d63f9ae04...|            14|
|341dc4ea81fabe7de...|8a8c2e93d6ec33bb9...|            28|
|3f71c7ea78bd7df67...|69a5de2d98d4729f8...|             

In [120]:
diff_delievery.count()

99441

In [ ]:
diff_delievery.filter(~col('diff_delievery').isNull()).count()

96476

In [124]:
tmp_df = diff_delievery.withColumn('is_late', diff_delievery['diff_delievery'] < 0)
tmp_df.filter(tmp_df['is_late']).show()

+--------------------+--------------------+--------------+-------+
|            order_id|         customer_id|diff_delievery|is_late|
+--------------------+--------------------+--------------+-------+
|2bd2241ff5bd59887...|adfc29e0c38c693ae...|            -8|   true|
|f5c7fcc9933ae8861...|d70e1d82169a6e535...|           -13|   true|
|226bdeb6252bfee90...|f344a641af9aeeb5d...|           -11|   true|
|d3cb6dffcf96cec44...|1dd1f1efccf6a33c8...|            -6|   true|
|5f862e4fe640e6611...|65a6a03c07caddea4...|            -3|   true|
|ec97666defb2e1fad...|4abad51a71113d2de...|            -3|   true|
|5c1ae73856bf1d099...|d5695e900ecca93e1...|            -3|   true|
|04b9ffa87087a2450...|fb26eff42e25a6ee6...|            -5|   true|
|376a342d70faa7a73...|8563b07a84656ff32...|           -15|   true|
|58b198357d2b68044...|8d7cf6fc4668c4885...|            -2|   true|
|93b94a19e5546ce4e...|d7dd2b2a9644847e0...|           -20|   true|
|649b01370e30b79f0...|9dd3d654d1cc127f5...|            -3|   t

In [ ]:
orders_with_diff = orders.withColumn(
    'diff_estimated_and_delivered',
    datediff(orders['order_estimated_delivery_date'], orders['order_delivered_customer_date'])
    )
orders_with_diff = orders_with_diff.filter(~orders_with_diff['diff_estimated_and_delivered'].isNull())
orders_with_diff

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|diff_estimated_and_delivered|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------------+
|acce194856392f074...|7e20bf5ca92da6820...|   delivered|     2018-06-04 00:00:13|2018-06-05 00:35:10|         2018-06-05 13:24:00|          2018-06-16 15:20:55|          2018-07-18 00:00:00|                          32|
|1d067305b599c1e0d...|0489975a325480c9e...|   delivered|     2018-02-14 13:05:17|2018-02-14 13:15:38|         2018-02-20

In [115]:
orders_with_diff.filter(orders_with_diff['diff_estimated_and_delivered'] < 0).show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|diff_estimated_and_delivered|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------------+
|2bd2241ff5bd59887...|adfc29e0c38c693ae...|   delivered|     2018-01-10 19:38:12|2018-01-12 02:38:38|         2018-01-17 00:47:54|          2018-02-06 15:05:19|          2018-01-29 00:00:00|                          -8|
|f5c7fcc9933ae8861...|d70e1d82169a6e535...|   delivered|     2017-11-24 12:36:02|2017-11-24 14:56:37|         2017-11-29

In [ ]:
late_delivered = orders.filter(datediff("order_delivered_customer_date", "order_estimated_delivery_date") > 0)
late_delivered.filter(~late_delivered['order_delivered_customer_date'].isNull()).show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|2bd2241ff5bd59887...|adfc29e0c38c693ae...|   delivered|     2018-01-10 19:38:12|2018-01-12 02:38:38|         2018-01-17 00:47:54|          2018-02-06 15:05:19|          2018-01-29 00:00:00|
|f5c7fcc9933ae8861...|d70e1d82169a6e535...|   delivered|     2017-11-24 12:36:02|2017-11-24 14:56:37|         2017-11-29 20:28:49|          2018-01-04 18:19:53|          2017-12-22 00:00:00|
|226bdeb6252bfee90...|f344a641af9aeeb5d...|  

In [ ]:
orders.select(datediff(orders['order_estimated_delivery_date'], orders['order_estimated_delivery_date'])).show()

+----------------------------------------------------------------------+
|datediff(order_estimated_delivery_date, order_estimated_delivery_date)|
+----------------------------------------------------------------------+
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                                     0|
|                                                  

In [73]:
orders.filter(orders['order_delivered_customer_date'].isNull()).show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|6942b8da583c2f995...|52006a9383bf149a4...|     shipped|     2018-01-10 11:33:07|2018-01-11 02:32:30|         2018-01-11 19:39:23|                         NULL|          2018-02-07 00:00:00|
|5b0ee19a3125fc083...|97e5ea711c00ba543...|    canceled|     2018-03-05 13:56:34|2018-03-05 14:40:27|                        NULL|                         NULL|          2018-03-15 00:00:00|
|2a5509f085830eb1e...|6253abaa7f49a14ed...|  

In [4]:
orders.filter("order_id == 'acce194856392f074dbf9dada14d8d82'").show(truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |2018-06-04 00:00:13     |2018-06-05 00:35:10|2018-06-05 13:24:00         |2018-06-16 15:20:55          |2018-07-18 00:00:00          |
+--------------------------------+--------------------------------+------------+------------------------+-------------------+---------------

In [5]:
melted_orders = orders.selectExpr(
    "order_id", "customer_id", "order_status",
    "stack(5, 'order_purchase_timestamp', order_purchase_timestamp, 'order_approved_at', order_approved_at, 'order_delivered_carrier_date', order_delivered_carrier_date, 'order_delivered_customer_date', order_delivered_customer_date, 'order_estimated_delivery_date', order_estimated_delivery_date) as (status, timestamp)"
)
melted_orders.sort(col("timestamp").desc()).show(truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|order_id                        |customer_id                     |order_status|status                       |timestamp          |
+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|b059ee4de278302d550a3035c4cdb740|856336203359aa6a61bf3826f7d84c49|canceled    |order_estimated_delivery_date|2018-11-12 00:00:00|
|10a045cdf6a5650c21e9cfeb60384c16|a4b417188addbc05b26b72d5e44837a1|canceled    |order_estimated_delivery_date|2018-10-30 00:00:00|
|0ba4f7e89d304a02edca85087abfd14c|b5c1c56fe1ec3cec6893b98d90a339bd|delivered   |order_estimated_delivery_date|2018-10-25 00:00:00|
|869997fbe01f39d184956b5c6bccfdbe|55c9dad94ec1a2ba57998bdb376c230a|canceled    |order_estimated_delivery_date|2018-10-25 00:00:00|
|616fa7d4871b87832197b2a137a115d2|bf6181a85bbb4115736c0a8db1a53be3|canceled    |ord

In [12]:
melted_orders.select(col('order_status')).distinct().show()

+------------+
|order_status|
+------------+
|     shipped|
|    canceled|
|    invoiced|
|     created|
|   delivered|
| unavailable|
|  processing|
|    approved|
+------------+



In [28]:
delievered = melted_orders.filter(melted_orders["order_status"] == "delivered")
delievered.show(5, truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|order_id                        |customer_id                     |order_status|status                       |timestamp          |
+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_purchase_timestamp     |2018-06-04 00:00:13|
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_approved_at            |2018-06-05 00:35:10|
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_delivered_carrier_date |2018-06-05 13:24:00|
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_delivered_customer_date|2018-06-16 15:20:55|
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |ord

In [38]:
delievered_all_status_distinct_count = delievered.groupBy(delievered['order_id']).agg(count_distinct('status').alias('status_distinct_count'))
delievered_all_status_distinct_count.select(delievered_all_status_distinct_count['status_distinct_count']).distinct().show()

+---------------------+
|status_distinct_count|
+---------------------+
|                    5|
+---------------------+



In [42]:
all_status_distinct_count = melted_orders.groupBy(melted_orders['order_id']).agg(count_distinct(melted_orders['status']).alias('status_distinct'))
all_status_distinct_count.select(all_status_distinct_count['status_distinct']).distinct().show()

+---------------+
|status_distinct|
+---------------+
|              5|
+---------------+



In [48]:
melted_orders.orderBy('order_id', 'timestamp').show(truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|order_id                        |customer_id                     |order_status|status                       |timestamp          |
+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|00010242fe8c5a6d1ba2dd792cb16214|3ce436f183e68e07877b285a838db11a|delivered   |order_purchase_timestamp     |2017-09-13 08:59:02|
|00010242fe8c5a6d1ba2dd792cb16214|3ce436f183e68e07877b285a838db11a|delivered   |order_approved_at            |2017-09-13 09:45:35|
|00010242fe8c5a6d1ba2dd792cb16214|3ce436f183e68e07877b285a838db11a|delivered   |order_delivered_carrier_date |2017-09-19 18:34:16|
|00010242fe8c5a6d1ba2dd792cb16214|3ce436f183e68e07877b285a838db11a|delivered   |order_delivered_customer_date|2017-09-20 23:43:48|
|00010242fe8c5a6d1ba2dd792cb16214|3ce436f183e68e07877b285a838db11a|delivered   |ord

In [ ]:
cols = [_col for _col in melted_orders.columns if _col != 'timestamp']

time_split = melted_orders.select(
    *cols,
    year("timestamp").alias("year"),
    month("timestamp").alias("month"),
    dayofmonth("timestamp").alias("day"),
    hour("timestamp").alias("hour"),
    minute("timestamp").alias("minute"),
    second("timestamp").alias("second")
)
time_split.show(truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------------+----+-----+---+----+------+------+
|order_id                        |customer_id                     |order_status|status                       |year|month|day|hour|minute|second|
+--------------------------------+--------------------------------+------------+-----------------------------+----+-----+---+----+------+------+
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_purchase_timestamp     |2018|6    |4  |0   |0     |13    |
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_approved_at            |2018|6    |5  |0   |35    |10    |
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_delivered_carrier_date |2018|6    |5  |13  |24    |0     |
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_delivered_customer_date|2018|6    |16 |15  |

In [71]:
delievered_time = melted_orders.filter(col('status').isin(["order_estimated_delivery_date", "order_delivered_customer_date"]))
delievered_time.show(5, truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|order_id                        |customer_id                     |order_status|status                       |timestamp          |
+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_delivered_customer_date|2018-06-16 15:20:55|
|acce194856392f074dbf9dada14d8d82|7e20bf5ca92da68200643bda76c504c6|delivered   |order_estimated_delivery_date|2018-07-18 00:00:00|
|1d067305b599c1e0dceb3864056ea527|0489975a325480c9e385e9f135bb13c3|delivered   |order_delivered_customer_date|2018-03-09 21:52:36|
|1d067305b599c1e0dceb3864056ea527|0489975a325480c9e385e9f135bb13c3|delivered   |order_estimated_delivery_date|2018-03-09 00:00:00|
|6f841dde94727854eaff3f66432c80ea|a9c9532060c9d245f06526c633d2dfba|delivered   |ord

In [ ]:
delievered_time.groupBy('order_id').agg(delievered_time['status'] == 'order_estimated_delivery_date' - delievered_time['status'] == 'order_delivered_customer_date').fitelr ( > 0)

In [6]:
null_counts = melted_orders.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in melted_orders.columns
])

null_counts.show()

+--------+-----------+------------+------+---------+
|order_id|customer_id|order_status|status|timestamp|
+--------+-----------+------------+------+---------+
|       0|          0|           0|     0|     4908|
+--------+-----------+------------+------+---------+



In [7]:
melted_orders.filter(col('timestamp').isNull()).select(col('status')).distinct().show(truncate=False)

+-----------------------------+
|status                       |
+-----------------------------+
|order_delivered_carrier_date |
|order_approved_at            |
|order_delivered_customer_date|
+-----------------------------+



In [8]:
null_timestamp = melted_orders.filter(col('timestamp').isNull())
null_timestamp.orderBy('status').show()

+--------------------+--------------------+------------+-----------------+---------+
|            order_id|         customer_id|order_status|           status|timestamp|
+--------------------+--------------------+------------+-----------------+---------+
|0d3adebce4bebc1f8...|a7c87c25dbdded68c...|    canceled|order_approved_at|     NULL|
|252788350f9dad30b...|43c2e0c2bffdba58e...|    canceled|order_approved_at|     NULL|
|e04abd8149ef81b95...|2127dc6603ac33544...|   delivered|order_approved_at|     NULL|
|2babbb4b15e6d2dfe...|74bebaf46603f9340...|   delivered|order_approved_at|     NULL|
|672ab3776d5b25e58...|8c8b3a28c07f67148...|    canceled|order_approved_at|     NULL|
|2bb02fdf5637a68cd...|8c468917b27485762...|    canceled|order_approved_at|     NULL|
|b715c631fdd01580d...|14d0a56007d15cde2...|    canceled|order_approved_at|     NULL|
|ddaec6fff982b13e7...|68f4ad79cc0c2ad06...|    canceled|order_approved_at|     NULL|
|2663ce3526465b7ee...|032a544bbe984490f...|    canceled|order_app

In [9]:
melted_orders.filter(col('order_status') == "canceled").show(truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|order_id                        |customer_id                     |order_status|status                       |timestamp          |
+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_purchase_timestamp     |2018-03-05 13:56:34|
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_approved_at            |2018-03-05 14:40:27|
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_delivered_carrier_date |NULL               |
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_delivered_customer_date|NULL               |
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |ord

In [10]:
melted_orders.filter(col('order_id') == "5b0ee19a3125fc083348baa76d816ed0").show(truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|order_id                        |customer_id                     |order_status|status                       |timestamp          |
+--------------------------------+--------------------------------+------------+-----------------------------+-------------------+
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_purchase_timestamp     |2018-03-05 13:56:34|
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_approved_at            |2018-03-05 14:40:27|
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_delivered_carrier_date |NULL               |
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |order_delivered_customer_date|NULL               |
|5b0ee19a3125fc083348baa76d816ed0|97e5ea711c00ba54392bf866a7d5f1e0|canceled    |ord